In [1]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotnine import *
from datetime import datetime, timedelta
from tqdm import tqdm

Load Data

In [20]:
fold = 5
data_train=pd.read_csv(f'share_info/5-fold/train_fold{fold}.csv')
data_test=pd.read_csv(f'share_info/5-fold/test_fold{fold}.csv')

# Add feature

Time stamp

In [21]:
data_train['DATETIME'] = pd.DatetimeIndex( data_train['DATETIME'])
data_test['DATETIME'] = pd.DatetimeIndex( data_test['DATETIME'])

In [22]:
data_train['month'] = pd.DatetimeIndex( data_train['DATETIME']).month
data_train['day'] = pd.DatetimeIndex( data_train['DATETIME']).day
data_train['hour'] = pd.DatetimeIndex( data_train['DATETIME']).hour
data_train['minute'] = pd.DatetimeIndex( data_train['DATETIME']).minute

data_test['month'] = pd.DatetimeIndex( data_test['DATETIME']).month
data_test['day'] = pd.DatetimeIndex( data_test['DATETIME']).day
data_test['hour'] = pd.DatetimeIndex( data_test['DATETIME']).hour
data_test['minute'] = pd.DatetimeIndex( data_test['DATETIME']).minute

EMA

In [23]:
data_train['EMA_08'] = data_train['I(t)'].ewm(alpha=0.8).mean()
data_test['EMA_08'] = data_test['I(t)'].ewm(alpha=0.8).mean()

I_clear sky

In [24]:
# Iclr.ashrae = K.*exp(-B./sza);  K = 1663.52224574355; B = 0.739550805574049;
# coefs. are from fitting with selected clearsky data
def clearsky_cal(x):
    if x > 0:
        return 1663.52225*2.71828**(-0.73955/x)
    else:
        return 0

In [25]:
for i in range(1,9):
    data_train['I_clr(t+{})'.format(i)] = data_train['sza(t+{})'.format(i)].apply(clearsky_cal)
    data_test['I_clr(t+{})'.format(i)] = data_test['sza(t+{})'.format(i)].apply(clearsky_cal)

# Prepare X, y

In [26]:
feature_cols = ['I(t)', 'I(t-1)','I(t-2)', 'I(t-3)', 'I(t-4)', 'I(t-5)'
                , 'I(t-6)', 'I(t-7)', \
                'I^(d-1)(t+1)', 'I^(d-1)(t+2)', 'I^(d-1)(t+3)','I^(d-1)(t+4)',\
                'I^(d-1)(t+5)', 'I^(d-1)(t+6)', 'I^(d-1)(t+7)','I^(d-1)(t+8)', \
                'sza(t+1)','sza(t+2)','sza(t+3)','sza(t+4)','sza(t+5)','sza(t+6)','sza(t+7)','sza(t+8)' \
               , 'EMA_08', 'RH(t)',
               'I_clr(t+1)','I_clr(t+2)','I_clr(t+3)','I_clr(t+4)','I_clr(t+5)','I_clr(t+6)','I_clr(t+7)',
                'I_clr(t+8)']

In [27]:
targets = ['I(t+1)','I(t+2)','I(t+3)','I(t+4)','I(t+5)','I(t+6)','I(t+7)','I(t+8)']

In [28]:
data_train = data_train.set_index('DATETIME')
data_train_dropna = data_train[feature_cols+targets].dropna()

In [29]:
data_test = data_test.set_index('DATETIME')
data_test_dropna = data_test[feature_cols+targets+['time']].dropna()

# Model

In [30]:
from sklearn import preprocessing
def normalizer_std(X):
    scaler = preprocessing.StandardScaler().fit(X)
    return scaler

In [41]:
def pre_processing(train_data,test_data,feature,label, std = True):
    
    x_train = train_data[feature].values
    y_train = train_data[label].values 
    
    x_test = test_data[feature].values
    y_test = test_data[label].values
    
    if std :
        x_scaler = normalizer_std(x_train)
        x_train = x_scaler.transform(x_train)
        x_test = x_scaler.transform(x_test)
        
    return x_train, x_test, y_train, y_test

In [32]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR 
from sklearn.linear_model import LinearRegression

## Prepare feature for each model
morning, midday, evening

## morning 6:00 - 9:30
<p> feature - focus on feature that indicate trend at the time of forecasts Iclr(t+k), I(d-1)(t+k), sza(t+k)
<p>I(t+1) : execution time = 5.30 to 9.00 -- include I(t)
<p>I(t+2) : execution time = 5.30 to 8.30
<p>I(t+3) : execution time = 5.30 to 8.00
<p>I(t+4) : execution time = 5.30 to 7.30
<p>I(t+5) : execution time = 5.30 to 7.00
<p>I(t+6) : execution time = 5.30 to 6.30
<p>I(t+7) : execution time = 5.30 to 6.00
<p>I(t+8) : execution time = 5.30

In [33]:
labels = ['I(t+1)', 'I(t+2)', 'I(t+3)', 'I(t+4)', 'I(t+5)', 'I(t+6)', 'I(t+7)','I(t+8)']
feature_dict_m = dict()
label_dict_m = dict()
for i in range(8):
    feature_dict_m['feature_{}'.format(i+1)] =   [f'sza(t+{i+1})',f'I^(d-1)(t+{i+1})',f'I_clr(t+{i+1})', 'EMA_08']
    label_dict_m['label_{}'.format(i+1)] = [labels[i]]
feature_dict_m['feature_1'].append('I(t)')

In [34]:
train_m1 = data_train.between_time('5:30','9:00')[feature_dict_m[f'feature_1'] + label_dict_m[f'label_1']]
train_m2 = data_train.between_time('5:30','8:30')[feature_dict_m[f'feature_2'] + label_dict_m[f'label_2']]
train_m3 = data_train.between_time('5:30','8:00')[feature_dict_m[f'feature_3'] + label_dict_m[f'label_3']]
train_m4 = data_train.between_time('5:30','7:30')[feature_dict_m[f'feature_4'] + label_dict_m[f'label_4']]
train_m5 = data_train.between_time('5:30','7:00')[feature_dict_m[f'feature_5'] + label_dict_m[f'label_5']]
train_m6 = data_train.between_time('5:30','6:30')[feature_dict_m[f'feature_6'] + label_dict_m[f'label_6']]
train_m7 = data_train.between_time('5:30','6:00')[feature_dict_m[f'feature_7'] + label_dict_m[f'label_7']]
train_m8 = data_train.between_time('5:30','5:30')[feature_dict_m[f'feature_8'] + label_dict_m[f'label_8']]

In [35]:
test_m1 = data_test.between_time('5:30','9:00')[feature_dict_m[f'feature_1'] + label_dict_m[f'label_1']]
test_m2 = data_test.between_time('5:30','8:30')[feature_dict_m[f'feature_2'] + label_dict_m[f'label_2']]
test_m3 = data_test.between_time('5:30','8:00')[feature_dict_m[f'feature_3'] + label_dict_m[f'label_3']]
test_m4 = data_test.between_time('5:30','7:30')[feature_dict_m[f'feature_4'] + label_dict_m[f'label_4']]
test_m5 = data_test.between_time('5:30','7:00')[feature_dict_m[f'feature_5'] + label_dict_m[f'label_5']]
test_m6 = data_test.between_time('5:30','6:30')[feature_dict_m[f'feature_6'] + label_dict_m[f'label_6']]
test_m7 = data_test.between_time('5:30','6:00')[feature_dict_m[f'feature_7'] + label_dict_m[f'label_7']]
test_m8 = data_test.between_time('5:30','5:30')[feature_dict_m[f'feature_8'] + label_dict_m[f'label_8']]

In [42]:
yhat_train_m = []
yhat_test_m = []
idx_train_m = []
idx_test_m = []
y_train_m = []
y_test_m= []

train_list_m = [train_m1, train_m2, train_m3, train_m4, train_m5, train_m6, train_m7, train_m8]
test_list_m = [test_m1, test_m2, test_m3, test_m4, test_m5, test_m6, test_m7, test_m8]
i = 0
for train,test in tqdm(zip(train_list_m, test_list_m)):
    
    x_train, x_test, y_train, y_test = pre_processing(train,test,feature_dict_m[f'feature_{i+1}'],
                                                      label_dict_m[f'label_{i+1}'], std = False)
    reg = RandomForestRegressor(n_estimators = 1000, max_depth=10,
                                random_state = 1, min_samples_split=34, min_samples_leaf=16)
    reg.fit(x_train, y_train.ravel())


    yhat_train_m.append(reg.predict(x_train))
    yhat_test_m.append(reg.predict(x_test))
    
    y_train_m.append(y_train)
    y_test_m.append(y_test)
    
    idx_train_m.append(train.index)
    idx_test_m.append(test.index)
    i+=1


0it [00:00, ?it/s]
1it [00:08,  8.41s/it]
2it [00:14,  7.82s/it]
3it [00:20,  7.13s/it]
4it [00:25,  6.41s/it]
5it [00:28,  5.66s/it]
6it [00:32,  4.87s/it]
7it [00:34,  4.12s/it]
8it [00:36,  4.51s/it]


In [43]:
yhat_train_m1 = pd.DataFrame(yhat_train_m[0],columns=['I(t+1)'], index = idx_train_m[0])
yhat_train_m2 = pd.DataFrame(yhat_train_m[1],columns=['I(t+2)'], index = idx_train_m[1])
yhat_train_m3 = pd.DataFrame(yhat_train_m[2],columns=['I(t+3)'], index = idx_train_m[2])
yhat_train_m4 = pd.DataFrame(yhat_train_m[3],columns=['I(t+4)'], index = idx_train_m[3])
yhat_train_m5 = pd.DataFrame(yhat_train_m[4],columns=['I(t+5)'], index = idx_train_m[4])
yhat_train_m6 = pd.DataFrame(yhat_train_m[5],columns=['I(t+6)'], index = idx_train_m[5])
yhat_train_m7 = pd.DataFrame(yhat_train_m[6],columns=['I(t+7)'], index = idx_train_m[6])
yhat_train_m8 = pd.DataFrame(yhat_train_m[7],columns=['I(t+8)'], index = idx_train_m[7])

y_train_m1 = pd.DataFrame(y_train_m[0],columns=['I(t+1)'], index = idx_train_m[0])
y_train_m2 = pd.DataFrame(y_train_m[1],columns=['I(t+2)'], index = idx_train_m[1])
y_train_m3 = pd.DataFrame(y_train_m[2],columns=['I(t+3)'], index = idx_train_m[2])
y_train_m4 = pd.DataFrame(y_train_m[3],columns=['I(t+4)'], index = idx_train_m[3])
y_train_m5 = pd.DataFrame(y_train_m[4],columns=['I(t+5)'], index = idx_train_m[4])
y_train_m6 = pd.DataFrame(y_train_m[5],columns=['I(t+6)'], index = idx_train_m[5])
y_train_m7 = pd.DataFrame(y_train_m[6],columns=['I(t+7)'], index = idx_train_m[6])
y_train_m8 = pd.DataFrame(y_train_m[7],columns=['I(t+8)'], index = idx_train_m[7])

In [44]:
yhat_test_m1 = pd.DataFrame(yhat_test_m[0],columns=['I(t+1)'], index = idx_test_m[0])
yhat_test_m2 = pd.DataFrame(yhat_test_m[1],columns=['I(t+2)'], index = idx_test_m[1])
yhat_test_m3 = pd.DataFrame(yhat_test_m[2],columns=['I(t+3)'], index = idx_test_m[2])
yhat_test_m4 = pd.DataFrame(yhat_test_m[3],columns=['I(t+4)'], index = idx_test_m[3])
yhat_test_m5 = pd.DataFrame(yhat_test_m[4],columns=['I(t+5)'], index = idx_test_m[4])
yhat_test_m6 = pd.DataFrame(yhat_test_m[5],columns=['I(t+6)'], index = idx_test_m[5])
yhat_test_m7 = pd.DataFrame(yhat_test_m[6],columns=['I(t+7)'], index = idx_test_m[6])
yhat_test_m8 = pd.DataFrame(yhat_test_m[7],columns=['I(t+8)'], index = idx_test_m[7])

y_test_m1 = pd.DataFrame(y_test_m[0],columns=['I(t+1)'], index = idx_test_m[0])
y_test_m2 = pd.DataFrame(y_test_m[1],columns=['I(t+2)'], index = idx_test_m[1])
y_test_m3 = pd.DataFrame(y_test_m[2],columns=['I(t+3)'], index = idx_test_m[2])
y_test_m4 = pd.DataFrame(y_test_m[3],columns=['I(t+4)'], index = idx_test_m[3])
y_test_m5 = pd.DataFrame(y_test_m[4],columns=['I(t+5)'], index = idx_test_m[4])
y_test_m6 = pd.DataFrame(y_test_m[5],columns=['I(t+6)'], index = idx_test_m[5])
y_test_m7 = pd.DataFrame(y_test_m[6],columns=['I(t+7)'], index = idx_test_m[6])
y_test_m8 = pd.DataFrame(y_test_m[7],columns=['I(t+8)'], index = idx_test_m[7])

## afternoon 10:00 - 15.30
feature - focus on both feature that indicate trend Iclr(t+k), I(d-1)(t+k), sza(t+k) and dynamic I(t),RH(t),..
<p>I(t+1) : execution time = 9.30 to 15.00
<p>I(t+2) : execution time = 9.00 to 14.30
<p>I(t+3) : execution time = 8.30 to 14.00
<p>I(t+4) : execution time = 8.00 to 13.30
<p>I(t+5) : execution time = 7.30 to 13.00
<p>I(t+6) : execution time = 7.00 to 12.30
<p>I(t+7) : execution time = 6.30 to 12.00
<p>I(t+8) : execution time = 6.00 to 11.30

In [45]:
labels = ['I(t+1)', 'I(t+2)', 'I(t+3)', 'I(t+4)', 'I(t+5)', 'I(t+6)', 'I(t+7)','I(t+8)']
feature_dict_a = dict()
label_dict_a = dict()
for i in range(8):
    feature_dict_a[f'feature_{i+1}'] =   [f'sza(t+{i+1})',f'I^(d-1)(t+{i+1})',f'I_clr(t+{i+1})',
                                                  'EMA_08','I(t)', 'I(t-1)', 'I(t-2)', 'I(t-3)', 'I(t-4)']
    label_dict_a[f'label_{i+1}'] = [labels[i]]

In [46]:
train_a1 = data_train.between_time('9:30','15:00')[feature_dict_a[f'feature_1'] + label_dict_a[f'label_1']]
train_a2 = data_train.between_time('9:00','14:30')[feature_dict_a[f'feature_2'] + label_dict_a[f'label_2']]
train_a3 = data_train.between_time('8:30','14:00')[feature_dict_a[f'feature_3'] + label_dict_a[f'label_3']]
train_a4 = data_train.between_time('8:00','13:30')[feature_dict_a[f'feature_4'] + label_dict_a[f'label_4']]
train_a5 = data_train.between_time('7:30','13:00')[feature_dict_a[f'feature_5'] + label_dict_a[f'label_5']]
train_a6 = data_train.between_time('7:00','12:30')[feature_dict_a[f'feature_6'] + label_dict_a[f'label_6']]
train_a7 = data_train.between_time('6:30','12:00')[feature_dict_a[f'feature_7'] + label_dict_a[f'label_7']]
train_a8 = data_train.between_time('6:00','11:30')[feature_dict_a[f'feature_8'] + label_dict_a[f'label_8']]

In [47]:
test_a1 = data_test.between_time('9:30','15:00')[feature_dict_a[f'feature_1'] + label_dict_a[f'label_1']]
test_a2 = data_test.between_time('9:00','14:30')[feature_dict_a[f'feature_2'] + label_dict_a[f'label_2']]
test_a3 = data_test.between_time('8:30','14:00')[feature_dict_a[f'feature_3'] + label_dict_a[f'label_3']]
test_a4 = data_test.between_time('8:00','13:30')[feature_dict_a[f'feature_4'] + label_dict_a[f'label_4']]
test_a5 = data_test.between_time('7:30','13:00')[feature_dict_a[f'feature_5'] + label_dict_a[f'label_5']]
test_a6 = data_test.between_time('7:00','12:30')[feature_dict_a[f'feature_6'] + label_dict_a[f'label_6']]
test_a7 = data_test.between_time('6:30','12:00')[feature_dict_a[f'feature_7'] + label_dict_a[f'label_7']]
test_a8 = data_test.between_time('6:00','11:30')[feature_dict_a[f'feature_8'] + label_dict_a[f'label_8']]

In [50]:
yhat_train_a = []
yhat_test_a = []
idx_train_a = []
idx_test_a = []
y_train_a = []
y_test_a = []

train_list_a = [train_a1, train_a2, train_a3, train_a4, train_a5, train_a6, train_a7, train_a8]
test_list_a = [test_a1, test_a2, test_a3, test_a4, test_a5, test_a6, test_a7, test_a8]
i = 0
for train,test in tqdm(zip(train_list_a, test_list_a)):
    x_train, x_test, y_train, y_test = pre_processing(train,test,feature_dict_a[f'feature_{i+1}'],
                                                      label_dict_a[f'label_{i+1}'], std = False)
    reg = RandomForestRegressor(n_estimators = 1000, max_depth=10,
                                random_state = 1, min_samples_split=34, min_samples_leaf=16)
    reg.fit(x_train, y_train.ravel())
    
    yhat_train_a.append(reg.predict(x_train))
    yhat_test_a.append(reg.predict(x_test))
    
    y_train_a.append(y_train)
    y_test_a.append(y_test)   
    
    idx_train_a.append(train.index)
    idx_test_a.append(test.index)
    i+=1



0it [00:00, ?it/s]

1it [00:25, 25.73s/it]

2it [00:52, 26.03s/it]

3it [01:21, 26.78s/it]

4it [01:50, 27.54s/it]

5it [02:17, 27.45s/it]

6it [02:48, 28.37s/it]

7it [03:14, 27.72s/it]

8it [03:42, 27.78s/it]


In [52]:
yhat_train_a1 = pd.DataFrame(yhat_train_a[0],columns=['I(t+1)'], index = idx_train_a[0])
yhat_train_a2 = pd.DataFrame(yhat_train_a[1],columns=['I(t+2)'], index = idx_train_a[1])
yhat_train_a3 = pd.DataFrame(yhat_train_a[2],columns=['I(t+3)'], index = idx_train_a[2])
yhat_train_a4 = pd.DataFrame(yhat_train_a[3],columns=['I(t+4)'], index = idx_train_a[3])
yhat_train_a5 = pd.DataFrame(yhat_train_a[4],columns=['I(t+5)'], index = idx_train_a[4])
yhat_train_a6 = pd.DataFrame(yhat_train_a[5],columns=['I(t+6)'], index = idx_train_a[5])
yhat_train_a7 = pd.DataFrame(yhat_train_a[6],columns=['I(t+7)'], index = idx_train_a[6])
yhat_train_a8 = pd.DataFrame(yhat_train_a[7],columns=['I(t+8)'], index = idx_train_a[7])

y_train_a1 = pd.DataFrame(y_train_a[0],columns=['I(t+1)'], index = idx_train_a[0])
y_train_a2 = pd.DataFrame(y_train_a[1],columns=['I(t+2)'], index = idx_train_a[1])
y_train_a3 = pd.DataFrame(y_train_a[2],columns=['I(t+3)'], index = idx_train_a[2])
y_train_a4 = pd.DataFrame(y_train_a[3],columns=['I(t+4)'], index = idx_train_a[3])
y_train_a5 = pd.DataFrame(y_train_a[4],columns=['I(t+5)'], index = idx_train_a[4])
y_train_a6 = pd.DataFrame(y_train_a[5],columns=['I(t+6)'], index = idx_train_a[5])
y_train_a7 = pd.DataFrame(y_train_a[6],columns=['I(t+7)'], index = idx_train_a[6])
y_train_a8 = pd.DataFrame(y_train_a[7],columns=['I(t+8)'], index = idx_train_a[7])

In [53]:
yhat_test_a1 = pd.DataFrame(yhat_test_a[0],columns=['I(t+1)'], index = idx_test_a[0])
yhat_test_a2 = pd.DataFrame(yhat_test_a[1],columns=['I(t+2)'], index = idx_test_a[1])
yhat_test_a3 = pd.DataFrame(yhat_test_a[2],columns=['I(t+3)'], index = idx_test_a[2])
yhat_test_a4 = pd.DataFrame(yhat_test_a[3],columns=['I(t+4)'], index = idx_test_a[3])
yhat_test_a5 = pd.DataFrame(yhat_test_a[4],columns=['I(t+5)'], index = idx_test_a[4])
yhat_test_a6 = pd.DataFrame(yhat_test_a[5],columns=['I(t+6)'], index = idx_test_a[5])
yhat_test_a7 = pd.DataFrame(yhat_test_a[6],columns=['I(t+7)'], index = idx_test_a[6])
yhat_test_a8 = pd.DataFrame(yhat_test_a[7],columns=['I(t+8)'], index = idx_test_a[7])

y_test_a1 = pd.DataFrame(y_test_a[0],columns=['I(t+1)'], index = idx_test_a[0])
y_test_a2 = pd.DataFrame(y_test_a[1],columns=['I(t+2)'], index = idx_test_a[1])
y_test_a3 = pd.DataFrame(y_test_a[2],columns=['I(t+3)'], index = idx_test_a[2])
y_test_a4 = pd.DataFrame(y_test_a[3],columns=['I(t+4)'], index = idx_test_a[3])
y_test_a5 = pd.DataFrame(y_test_a[4],columns=['I(t+5)'], index = idx_test_a[4])
y_test_a6 = pd.DataFrame(y_test_a[5],columns=['I(t+6)'], index = idx_test_a[5])
y_test_a7 = pd.DataFrame(y_test_a[6],columns=['I(t+7)'], index = idx_test_a[6])
y_test_a8 = pd.DataFrame(y_test_a[7],columns=['I(t+8)'], index = idx_test_a[7])

## evening 16:00 - 18:00
feature - focus on  feature that indicate trend Iclr(t+k), I(d-1)(t+k), sza(t+k) 
<p>I(t+1) : execution time = 15.30 to 17.30
<p>I(t+2) : execution time = 15.00 to 17.30
<p>I(t+3) : execution time = 14.30 to 17.30
<p>I(t+4) : execution time = 14.00 to 17.30
<p>I(t+5) : execution time = 13.30 to 17.30
<p>I(t+6) : execution time = 13.00 to 17.30
<p>I(t+7) : execution time = 12.30 to 17.30
<p>I(t+8) : execution time = 12.00 to 17.30

In [54]:
labels = ['I(t+1)', 'I(t+2)', 'I(t+3)', 'I(t+4)', 'I(t+5)', 'I(t+6)', 'I(t+7)','I(t+8)']
feature_dict_e = dict()
label_dict_e = dict()
for i in range(8):
    feature_dict_e['feature_{}'.format(i+1)] =  [f'sza(t+{i+1})',f'I^(d-1)(t+{i+1})',f'I_clr(t+{i+1})', 'EMA_08', ]
    label_dict_e['label_{}'.format(i+1)] = [labels[i]]
feature_dict_e['feature_1'].append('I(t)')

In [55]:
train_e1 = data_train.between_time('15:30','17:30')[feature_dict_e[f'feature_1'] + label_dict_e[f'label_1']]
train_e2 = data_train.between_time('15:00','17:30')[feature_dict_e[f'feature_2'] + label_dict_e[f'label_2']]
train_e3 = data_train.between_time('14:30','17:30')[feature_dict_e[f'feature_3'] + label_dict_e[f'label_3']]
train_e4 = data_train.between_time('14:00','17:30')[feature_dict_e[f'feature_4'] + label_dict_e[f'label_4']]
train_e5 = data_train.between_time('13:30','17:30')[feature_dict_e[f'feature_5'] + label_dict_e[f'label_5']]
train_e6 = data_train.between_time('13:00','17:30')[feature_dict_e[f'feature_6'] + label_dict_e[f'label_6']]
train_e7 = data_train.between_time('12:30','17:30')[feature_dict_e[f'feature_7'] + label_dict_e[f'label_7']]
train_e8 = data_train.between_time('12:00','17:30')[feature_dict_e[f'feature_8'] + label_dict_e[f'label_8']]

In [56]:
test_e1 = data_test.between_time('15:30','17:30')[feature_dict_e[f'feature_1'] + label_dict_e[f'label_1']]
test_e2 = data_test.between_time('15:00','17:30')[feature_dict_e[f'feature_2'] + label_dict_e[f'label_2']]
test_e3 = data_test.between_time('14:30','17:30')[feature_dict_e[f'feature_3'] + label_dict_e[f'label_3']]
test_e4 = data_test.between_time('14:00','17:30')[feature_dict_e[f'feature_4'] + label_dict_e[f'label_4']]
test_e5 = data_test.between_time('13:30','17:30')[feature_dict_e[f'feature_5'] + label_dict_e[f'label_5']]
test_e6 = data_test.between_time('13:00','17:30')[feature_dict_e[f'feature_6'] + label_dict_e[f'label_6']]
test_e7 = data_test.between_time('12:30','17:30')[feature_dict_e[f'feature_7'] + label_dict_e[f'label_7']]
test_e8 = data_test.between_time('12:00','17:30')[feature_dict_e[f'feature_8'] + label_dict_e[f'label_8']]

In [57]:
yhat_train_e = []
yhat_test_e = []
idx_train_e = []
idx_test_e = []
y_train_e = []
y_test_e = []
 
train_list_e = [train_e1, train_e2, train_e3, train_e4, train_e5, train_e6, train_e7, train_e8]
test_list_e = [test_e1, test_e2, test_e3, test_e4, test_e5, test_e6, test_e7, test_e8]
i = 0
for train,test in tqdm(zip(train_list_e, test_list_e)):
    x_train, x_test, y_train, y_test = pre_processing(train,test,feature_dict_e[f'feature_{i+1}'],
                                                      label_dict_e[f'label_{i+1}'], std = False)
    reg = RandomForestRegressor(n_estimators = 1000, max_depth=10,
                                random_state = 1, min_samples_split=34, min_samples_leaf=16)
    reg.fit(x_train, y_train.ravel())
    
    yhat_train_e.append(reg.predict(x_train))
    yhat_test_e.append(reg.predict(x_test))
    
    y_train_e.append(y_train)
    y_test_e.append(y_test)
    
    idx_train_e.append(train.index)
    idx_test_e.append(test.index)
    i+=1



0it [00:00, ?it/s]

1it [00:06,  6.16s/it]

2it [00:12,  6.18s/it]

3it [00:18,  6.19s/it]

4it [00:26,  6.72s/it]

5it [00:36,  7.55s/it]

6it [00:44,  7.81s/it]

7it [00:55,  8.73s/it]

8it [01:06,  8.25s/it]


In [58]:
yhat_train_e1 = pd.DataFrame(yhat_train_e[0],columns=['I(t+1)'], index = idx_train_e[0])
yhat_train_e2 = pd.DataFrame(yhat_train_e[1],columns=['I(t+2)'], index = idx_train_e[1])
yhat_train_e3 = pd.DataFrame(yhat_train_e[2],columns=['I(t+3)'], index = idx_train_e[2])
yhat_train_e4 = pd.DataFrame(yhat_train_e[3],columns=['I(t+4)'], index = idx_train_e[3])
yhat_train_e5 = pd.DataFrame(yhat_train_e[4],columns=['I(t+5)'], index = idx_train_e[4])
yhat_train_e6 = pd.DataFrame(yhat_train_e[5],columns=['I(t+6)'], index = idx_train_e[5])
yhat_train_e7 = pd.DataFrame(yhat_train_e[6],columns=['I(t+7)'], index = idx_train_e[6])
yhat_train_e8 = pd.DataFrame(yhat_train_e[7],columns=['I(t+8)'], index = idx_train_e[7])

y_train_e1 = pd.DataFrame(y_train_e[0],columns=['I(t+1)'], index = idx_train_e[0])
y_train_e2 = pd.DataFrame(y_train_e[1],columns=['I(t+2)'], index = idx_train_e[1])
y_train_e3 = pd.DataFrame(y_train_e[2],columns=['I(t+3)'], index = idx_train_e[2])
y_train_e4 = pd.DataFrame(y_train_e[3],columns=['I(t+4)'], index = idx_train_e[3])
y_train_e5 = pd.DataFrame(y_train_e[4],columns=['I(t+5)'], index = idx_train_e[4])
y_train_e6 = pd.DataFrame(y_train_e[5],columns=['I(t+6)'], index = idx_train_e[5])
y_train_e7 = pd.DataFrame(y_train_e[6],columns=['I(t+7)'], index = idx_train_e[6])
y_train_e8 = pd.DataFrame(y_train_e[7],columns=['I(t+8)'], index = idx_train_e[7])

In [59]:
yhat_test_e1 = pd.DataFrame(yhat_test_e[0],columns=['I(t+1)'], index = idx_test_e[0])
yhat_test_e2 = pd.DataFrame(yhat_test_e[1],columns=['I(t+2)'], index = idx_test_e[1])
yhat_test_e3 = pd.DataFrame(yhat_test_e[2],columns=['I(t+3)'], index = idx_test_e[2])
yhat_test_e4 = pd.DataFrame(yhat_test_e[3],columns=['I(t+4)'], index = idx_test_e[3])
yhat_test_e5 = pd.DataFrame(yhat_test_e[4],columns=['I(t+5)'], index = idx_test_e[4])
yhat_test_e6 = pd.DataFrame(yhat_test_e[5],columns=['I(t+6)'], index = idx_test_e[5])
yhat_test_e7 = pd.DataFrame(yhat_test_e[6],columns=['I(t+7)'], index = idx_test_e[6])
yhat_test_e8 = pd.DataFrame(yhat_test_e[7],columns=['I(t+8)'], index = idx_test_e[7])

y_test_e1 = pd.DataFrame(y_test_e[0],columns=['I(t+1)'], index = idx_test_e[0])
y_test_e2 = pd.DataFrame(y_test_e[1],columns=['I(t+2)'], index = idx_test_e[1])
y_test_e3 = pd.DataFrame(y_test_e[2],columns=['I(t+3)'], index = idx_test_e[2])
y_test_e4 = pd.DataFrame(y_test_e[3],columns=['I(t+4)'], index = idx_test_e[3])
y_test_e5 = pd.DataFrame(y_test_e[4],columns=['I(t+5)'], index = idx_test_e[4])
y_test_e6 = pd.DataFrame(y_test_e[5],columns=['I(t+6)'], index = idx_test_e[5])
y_test_e7 = pd.DataFrame(y_test_e[6],columns=['I(t+7)'], index = idx_test_e[6])
y_test_e8 = pd.DataFrame(y_test_e[7],columns=['I(t+8)'], index = idx_test_e[7])

## get yhat dataframe : merge all yhat

In [60]:
from functools import reduce

In [79]:
yhat_train1 = pd.concat([yhat_train_m1,yhat_train_a1,yhat_train_e1]).sort_index()
yhat_train2 = pd.concat([yhat_train_m2,yhat_train_a2,yhat_train_e2]).sort_index()
yhat_train3 = pd.concat([yhat_train_m3,yhat_train_a3,yhat_train_e3]).sort_index()
yhat_train4 = pd.concat([yhat_train_m4,yhat_train_a4,yhat_train_e4]).sort_index()
yhat_train5 = pd.concat([yhat_train_m5,yhat_train_a5,yhat_train_e5]).sort_index()
yhat_train6 = pd.concat([yhat_train_m6,yhat_train_a6,yhat_train_e6]).sort_index()
yhat_train7 = pd.concat([yhat_train_m7,yhat_train_a7,yhat_train_e7]).sort_index()
yhat_train8 = pd.concat([yhat_train_m8,yhat_train_a8,yhat_train_e8]).sort_index()

y_train1 = pd.concat([y_train_m1,y_train_a1,y_train_e1]).sort_index()
y_train2 = pd.concat([y_train_m2,y_train_a2,y_train_e2]).sort_index()
y_train3 = pd.concat([y_train_m3,y_train_a3,y_train_e3]).sort_index()
y_train4 = pd.concat([y_train_m4,y_train_a4,y_train_e4]).sort_index()
y_train5 = pd.concat([y_train_m5,y_train_a5,y_train_e5]).sort_index()
y_train6 = pd.concat([y_train_m6,y_train_a6,y_train_e6]).sort_index()
y_train7 = pd.concat([y_train_m7,y_train_a7,y_train_e7]).sort_index()
y_train8 = pd.concat([y_train_m8,y_train_a8,y_train_e8]).sort_index()

yhat_test1 = pd.concat([yhat_test_m1, yhat_test_a1, yhat_test_e1]).sort_index()
yhat_test2 = pd.concat([yhat_test_m2, yhat_test_a2, yhat_test_e2]).sort_index()
yhat_test3 = pd.concat([yhat_test_m3, yhat_test_a3, yhat_test_e3]).sort_index()
yhat_test4 = pd.concat([yhat_test_m4, yhat_test_a4, yhat_test_e4]).sort_index()
yhat_test5 = pd.concat([yhat_test_m5, yhat_test_a5, yhat_test_e5]).sort_index()
yhat_test6 = pd.concat([yhat_test_m6, yhat_test_a6, yhat_test_e6]).sort_index()
yhat_test7 = pd.concat([yhat_test_m7, yhat_test_a7, yhat_test_e7]).sort_index()
yhat_test8 = pd.concat([yhat_test_m8, yhat_test_a8, yhat_test_e8]).sort_index()

y_test1 = pd.concat([y_test_m1, y_test_a1, y_test_e1]).sort_index()
y_test2 = pd.concat([y_test_m2, y_test_a2, y_test_e2]).sort_index()
y_test3 = pd.concat([y_test_m3, y_test_a3, y_test_e3]).sort_index()
y_test4 = pd.concat([y_test_m4, y_test_a4, y_test_e4]).sort_index()
y_test5 = pd.concat([y_test_m5, y_test_a5, y_test_e5]).sort_index()
y_test6 = pd.concat([y_test_m6, y_test_a6, y_test_e6]).sort_index()
y_test7 = pd.concat([y_test_m7, y_test_a7, y_test_e7]).sort_index()
y_test8 = pd.concat([y_test_m8, y_test_a8, y_test_e8]).sort_index()

In [80]:
dfhat_train = [yhat_train1, yhat_train2, yhat_train3, yhat_train4, yhat_train5, yhat_train6, yhat_train7, yhat_train8]
dfhat_test = [yhat_test1, yhat_test2, yhat_test3, yhat_test4, yhat_test5, yhat_test6, yhat_test7, yhat_test8]

df_train = [y_train1, y_train2, y_train3, y_train4, y_train5, y_train6, y_train7, y_train8]
df_test = [y_test1, y_test2, y_test3, y_test4, y_test5, y_test6, y_test7, y_test8]

y_hat_train = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), dfhat_train)
y_hat_test = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), dfhat_test)

y_train = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), df_train)
y_test = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), df_test)

In [81]:
y_hat_train.dropna(inplace = True)
y_hat_test.dropna(inplace = True)
y_train.dropna(inplace = True)
y_test.dropna(inplace = True)

# Error analysis

In [82]:
def RMSETable_1( y_train, y_hat_train, y_test, y_hat_test, targets, train = True ):
    time_list = ['5:30','6:00','6:30','7:00','7:30','8:00','8:30','9:00','9:30','10:00','10:30','11:00','11:30',\
            '12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30','16:00','16:30','17:00', '17:30']
    table = pd.DataFrame({'Execution Time': time_list})
    for i in targets:
        temp1 = []
        temp2 = []
        for j in time_list:
            temp1.append (np.sqrt(MSE(y_train[i].between_time(j, j), y_hat_train[i].between_time(j, j))))
            temp2.append (np.sqrt(MSE(y_test[i].between_time(j, j), y_hat_test[i].between_time(j, j))))
        if train == True:
            table[i+'RMSE_train'] = temp1
        table[i+'RMSE_test'] = temp2
    return table

In [83]:
def RMSETable_2( y_hat_train, y_hat_test, train_on = True):
    time_list = ['6:00','6:30','7:00','7:30','8:00','8:30','9:00','9:30','10:00','10:30','11:00','11:30',\
            '12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30','16:00','16:30','17:00', '17:30']
    table = pd.DataFrame({'Time of forecasted value I(t)': time_list})
    for i in range(1, 9):
        temp1 = []
        temp2 = []
        for j in time_list:
            if train_on == True:
                temp1.append (np.sqrt(np.nanmean( (y_hat_train['Step-{}'.format(i)].between_time(j, j) - y_hat_train['I_mea'].between_time(j, j))**2)))
            temp2.append (np.sqrt(np.nanmean( (y_hat_test['Step-{}'.format(i)].between_time(j, j) - y_hat_test['I_mea'].between_time(j, j))**2)))
        if train_on == True:
            table['Step-{}_train'.format(i)] = temp1
        table['Step-{}_test'.format(i)] = temp2
    return table

In [84]:
def RMSESummary_1( y_train, y_hat_train, y_test, y_hat_test, targets):
    table = pd.DataFrame({'Step': targets})
    train=[]
    test=[]
    for i in range(1,9):
        train.append (np.sqrt(MSE(y_train['I(t+{})'.format(i)], y_hat_train['I(t+{})'.format(i)])))
        test.append (np.sqrt(MSE(y_test['I(t+{})'.format(i)], y_hat_test['I(t+{})'.format(i)])))
    table['train'] = train
    table['test'] = test      
    return table

In [85]:
def table_transform( table_input ):
    table_input['DATE'] = table_input.index.date
    table = pd.DataFrame({'Time of forecasted value': table_input.index})
    for i in range(1,9):
        table['Step-{}'.format(i)] = table_input.groupby('DATE').shift(i)['I(t+{})'.format(i)].values
    table = table.set_index('Time of forecasted value')
    return table

In [86]:
RMSE_table_1 = RMSETable_1(y_train, y_hat_train, y_test, y_hat_test, targets, train =False)
RMSE_table_sum = RMSESummary_1(y_train, y_hat_train, y_test, y_hat_test, targets)

In [87]:
Y_train_T = table_transform(y_train)
y_hat_train_T = table_transform(y_hat_train)
Y_test_T = table_transform(y_test)
y_hat_test_T = table_transform(y_hat_test)

In [88]:
Y_train_T['I_mea'] = data_train['I(t)']
y_hat_train_T['I_mea'] = data_train['I(t)']
Y_test_T['I_mea'] = data_test['I(t)']
y_hat_test_T['I_mea'] = data_test['I(t)']

In [89]:
RMSE_table_2 = RMSETable_2(y_hat_train_T, y_hat_test_T, train_on =False )

/Users/admin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


Error table

# I2P

In [91]:
df = data_train[['I(t)', 'P(t)']]

In [92]:
df['I(t)^2'] = df['I(t)']**2
df['I(t)^3'] = df['I(t)']**3
df['P(t)'] = df['P(t)']
df = df.dropna()


/Users/admin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/admin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/admin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [93]:
from sklearn.linear_model import LinearRegression

In [94]:
model_I2P  = LinearRegression().fit(df[['I(t)', 'I(t)^2', 'I(t)^3']], df[['P(t)']])

In [95]:
model_I2P.coef_ 

array([[ 4.48970314e-03,  7.06825811e-06, -6.35211588e-09]])

# Convert I to P

In [96]:
convert_list =['P(t+1)', 'P(t+2)','P(t+3)','P(t+4)','P(t+5)','P(t+6)','P(t+7)','P(t+8)']

In [97]:
#creat I^2 I^3
for i in range(1, 9):
    y_hat_test['I(t+{})^2'.format(i)] =  y_hat_test['I(t+{})'.format(i)]**2
    y_hat_test['I(t+{})^3'.format(i)] =  y_hat_test['I(t+{})'.format(i)]**3
    y_hat_train['I(t+{})^2'.format(i)] =  y_hat_train['I(t+{})'.format(i)]**2
    y_hat_train['I(t+{})^3'.format(i)] =  y_hat_train['I(t+{})'.format(i)]**3

In [98]:
for i in range(1, 9):
    y_hat_test['P(t+{})'.format(i)] = model_I2P.predict( y_hat_test[ ['I(t+{})'.format(i), 'I(t+{})^2'.format(i), 'I(t+{})^3'.format(i)] ] )
    y_hat_train['P(t+{})'.format(i)] = model_I2P.predict( y_hat_train[ ['I(t+{})'.format(i), 'I(t+{})^2'.format(i), 'I(t+{})^3'.format(i)] ] )

In [99]:
data_test_full = y_hat_test.merge(data_test[convert_list], left_index=True, right_index = True )
data_train_full = y_hat_train.merge(data_train[convert_list], left_index=True, right_index = True )

# Generate NRMSE Table

In [100]:
def NRMSE_P_Table_1( data_train, data_test, targets, train = True ):
    time_list = ['5:30','6:00','6:30','7:00','7:30','8:00','8:30','9:00','9:30','10:00','10:30','11:00','11:30',\
            '12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30','16:00','16:30','17:00', '17:30']
    data_train = data_train.dropna()
    data_test = data_test.dropna()
    table = pd.DataFrame({'Execution Time': time_list})
    for i in targets:
        temp1 = []
        temp2 = []
        for j in time_list:
            temp1.append ( np.sqrt(MSE(data_train[i+'_x'].between_time(j, j), data_train[i+'_y'].between_time(j, j)))/8*100 )
            temp2.append (np.sqrt(MSE(data_test[i+'_x'].between_time(j, j), data_test[i+'_y'].between_time(j, j)))/8*100)
        if train == True:
            table[i+'NRMSE_train'] = temp1
        table[i+'NRMSE_test'] = temp2
    return table

In [101]:
def table_transformP( table_input):
    table_input['DATE'] = table_input.index.date
    table = pd.DataFrame({'Time of forecasted value P(t)': table_input.index})
    table['P_mea'] = table_input.groupby('DATE').shift(1)['P(t+1)_y'.format(1)].values
    for i in range(1,9):
        table['Step-{}'.format(i)] = table_input.groupby('DATE').shift(i)['P(t+{})_x'.format(i)].values
    table = table.set_index('Time of forecasted value P(t)')
    return table

In [102]:
def NRMSE_P_Table_2( data_train, data_test, train_on = True ):
    time_list = ['6:00','6:30','7:00','7:30','8:00','8:30','9:00','9:30','10:00','10:30','11:00','11:30',\
            '12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30','16:00','16:30','17:00', '17:30']
    table = pd.DataFrame({'Time of forecasted values': time_list})
    for i in range(1,9):
        temp1 = []
        temp2 = []
        for j in time_list:
            if train_on == True:
                temp1.append (np.sqrt(np.nanmean( (data_train['Step-{}'.format(i)].between_time(j, j) - data_train['P_mea'].between_time(j, j) )**2))/8*100)
            temp2.append (np.sqrt(np.nanmean( (data_test['Step-{}'.format(i)].between_time(j, j) - data_test['P_mea'].between_time(j, j))**2))/8*100)
        if train_on == True:
            table['Step-{}_train'.format(i)] = temp1
        table['Step-{}_test'.format(i)] = temp2
    return table

In [103]:
def NRMSESummary_P( data_train, data_test, targets, train_on = True):
    table = pd.DataFrame({'Step': targets})
    data_train = data_train.dropna()
    data_test = data_test.dropna()
    train=[]
    test=[]
    for i in range(1,9):
        train.append (np.sqrt(MSE(data_train['P(t+{})_x'.format(i)], data_train['P(t+{})_y'.format(i)]))/8*100)
        test.append (np.sqrt(MSE(data_test['P(t+{})_x'.format(i)], data_test['P(t+{})_y'.format(i)]))/8*100)
    if train_on == True:
        table['train'] = train
    table['test'] = test      
    return table

In [104]:
NRMSE_P_table_1 = NRMSE_P_Table_1( data_train_full, data_test_full, convert_list, train = False )

In [105]:
NRMSE_P_table_2 = NRMSE_P_Table_2( table_transformP( data_train_full ) , table_transformP( data_test_full), train_on = False)

/Users/admin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


In [107]:
NRMSE_table_sum = NRMSESummary_P(data_train_full, data_test_full, convert_list)

In [74]:
tablePformat2 = table_transformP( data_test_full)

## Save file